Building the SmartTag models
===


## First time use

In [ ]:
copy neo4j database to neo4j installation

In [1]:
cd ~/Documents/code/py-smtag

In [3]:
neo4j stop

Stopping Neo4j.... stopped


In [ ]:
cd corpora

In [ ]:
unzip graph.db.zip

In [ ]:
cp graph.db.zip ../../neo4j/data/databases/

In [4]:
neo4j start

Starting Neo4j.
Started neo4j (pid 65494). It is available at http://localhost:7474/
There may be a short delay until the server is ready.
See /Users/lemberger/Documents/code/neo4j/logs/neo4j.log for current status.


In [ ]:
cd ..

## Initial checks

In [2]:
source .venv/bin/activate

(.venv) 

: 1

In [3]:
python setup.py install

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/distutils/dist.py:261: UserWarning: Unknown distribution option: 'long_description_content_type'
  warnings.warn(msg)
running install
running bdist_egg
running egg_info
writing smtag.egg-info/PKG-INFO
writing dependency_links to smtag.egg-info/dependency_links.txt
writing entry points to smtag.egg-info/entry_points.txt
writing requirements to smtag.egg-info/requires.txt
writing top-level names to smtag.egg-info/top_level.txt
reading manifest file 'smtag.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'smtag.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.6-intel/egg
running install_lib
running build_py
copying smtag/train/scanner.py -> build/lib/smtag/train
creating build/bdist.macosx-10.6-intel/egg
creating build/bdist.macosx-10.6-intel/egg/smtag
copying build/lib/smtag/__init__.py -> build/bdist.macosx-10.6-intel/egg/smtag
copying build/lib/smtag/command_line.py 

byte-compiling build/bdist.macosx-10.6-intel/egg/smtag/datagen/neo2leg~backup.py to neo2leg~backup.cpython-36.pyc
byte-compiling build/bdist.macosx-10.6-intel/egg/smtag/datagen/neo2xml.py to neo2xml.cpython-36.pyc
byte-compiling build/bdist.macosx-10.6-intel/egg/smtag/datagen/sdgraph2th.py to sdgraph2th.cpython-36.pyc
byte-compiling build/bdist.macosx-10.6-intel/egg/smtag/datagen/sdgraph2th~backup.py to sdgraph2th~backup.cpython-36.pyc
byte-compiling build/bdist.macosx-10.6-intel/egg/smtag/predict/__init__.py to __init__.cpython-36.pyc
byte-compiling build/bdist.macosx-10.6-intel/egg/smtag/predict/binarize.py to binarize.cpython-36.pyc
byte-compiling build/bdist.macosx-10.6-intel/egg/smtag/predict/engine.py to engine.cpython-36.pyc
byte-compiling build/bdist.macosx-10.6-intel/egg/smtag/predict/predictor.py to predictor.cpython-36.pyc
byte-compiling build/bdist.macosx-10.6-intel/egg/smtag/predict/serializer.py to serializer.cpython-36.pyc
byte-compiling build/bdist.macosx-10.6-intel/egg

Installing easy_install-3.6 script to /Users/lemberger/Documents/code/py-smtag/.venv/bin

Using /Users/lemberger/Documents/code/py-smtag/.venv/lib/python3.6/site-packages
Searching for jedi==0.12.1
Best match: jedi 0.12.1
Adding jedi 0.12.1 to easy-install.pth file

Using /Users/lemberger/Documents/code/py-smtag/.venv/lib/python3.6/site-packages
Searching for simplegeneric==0.8.1
Best match: simplegeneric 0.8.1
Adding simplegeneric 0.8.1 to easy-install.pth file

Using /Users/lemberger/Documents/code/py-smtag/.venv/lib/python3.6/site-packages
Searching for decorator==4.3.0
Best match: decorator 4.3.0
Adding decorator 4.3.0 to easy-install.pth file

Using /Users/lemberger/Documents/code/py-smtag/.venv/lib/python3.6/site-packages
Searching for pexpect==4.6.0
Best match: pexpect 4.6.0
Adding pexpect 4.6.0 to easy-install.pth file

Using /Users/lemberger/Documents/code/py-smtag/.venv/lib/python3.6/site-packages
Searching for traitlets==4.3.2
Best match: traitlets 4.3.2
Adding traitlets 4.3

Best match: pandocfilters 1.4.2
Adding pandocfilters 1.4.2 to easy-install.pth file

Using /Users/lemberger/Documents/code/py-smtag/.venv/lib/python3.6/site-packages
Searching for defusedxml==0.5.0
Best match: defusedxml 0.5.0
Adding defusedxml 0.5.0 to easy-install.pth file

Using /Users/lemberger/Documents/code/py-smtag/.venv/lib/python3.6/site-packages
Searching for mistune==0.8.3
Best match: mistune 0.8.3
Adding mistune 0.8.3 to easy-install.pth file

Using /Users/lemberger/Documents/code/py-smtag/.venv/lib/python3.6/site-packages
Searching for bleach==1.5.0
Best match: bleach 1.5.0
Adding bleach 1.5.0 to easy-install.pth file

Using /Users/lemberger/Documents/code/py-smtag/.venv/lib/python3.6/site-packages
Searching for certifi==2018.8.13
Best match: certifi 2018.8.13
Adding certifi 2018.8.13 to easy-install.pth file

Using /Users/lemberger/Documents/code/py-smtag/.venv/lib/python3.6/site-packages
Searching for idna==2.7
Best match: idna 2.7
Adding idna 2.7 to easy-install.pth fil

: 1

In [4]:
ls

LICENSE.txt	common		dist		rack		smtag
MANIFEST.in	corpora		docs		runs		smtag.egg-info
README.md	data		log		scans		test
__pycache__	data4th		models		setup.cfg	train
build		datagen		predict		setup.py
(.venv) 

: 1

In [5]:
ls models

geneprod.zip
(.venv) 

: 1

In [7]:
ls data

NCBI_disease	demo_xml	gene_protein	small_molecule
(.venv) 

: 1

In [8]:
ls data4th

5X_L1200_gene_protein_test
5X_L1200_gene_protein_test.zip
5X_L1200_small_molecule_test.zip
5X_L1200_small_molecule_train.prov
5X_L1200_small_molecule_train.pyth
5X_L1200_small_molecule_train.txt
5X_L1200_small_molecule_train.zip
5X_L1200_small_molecule_train_textcoded.pyth
NCBI_disease_test_test.zip
NCBI_disease_train_train.zip
demo_xml_train.prov
demo_xml_train.pyth
demo_xml_train.txt
demo_xml_train.zip
demo_xml_train_textcoded.pyth
(.venv) 

: 1

In [ ]:
smtag-neo2xml --help

In [ ]:
smtag-convert2th --help

In [ ]:
smtag-meta --help

In [ ]:
smtag-eval --help

In [ ]:
smtag-predict --help

## A. Data generation

### Small molecules

Enrich examples for small molecules (`-y`). Generate the data with `-5X` sampling of each example of length `-L1200` characters.

In [ ]:
smtag-neo2xml -l1000 -y molecule -f small_molecule

In [14]:
smtag-convert2th -X5 -L1200 -c small_molecule -f 5X_L1200_small_molecule


loading from: small_molecule/train
[############################################################] 100% ...loaded 99.xml
688 examples; desired length:1200

[############################################################] 100% ...sampling example 687
length of the 688 examples selected:
795 +/- 589 (min = 47, max = 5292)
Exec time for 'run': 64.704s
saving to 5X_L1200_small_molecule_train
saved 5X_L1200_small_molecule_train.pyth (size: 93843513)
saved 5X_L1200_small_molecule_train_textcoded.pyth (size: 136499513)
saved 5X_L1200_small_molecule_train.txt (size: 4289107)
saved 5X_L1200_small_molecule_train.prov (size: 40285)
(.venv) 

: 1

Prepare test set (`-T`) for later:

In [16]:
smtag-convert2th -X5 -L1200 -X5 -c small_molecule -f 5X_L1200_small_molecule -T


loading from: small_molecule/test
[############################################################] 100% ...loaded 99.xml
176 examples; desired length:1200

[############################################################] 100% ...sampling example 175
length of the 176 examples selected:
826 +/- 567 (min = 110, max = 3432)
Exec time for 'run': 17.226s
saving to 5X_L1200_small_molecule_test
saved 5X_L1200_small_molecule_test.pyth (size: 24006713)
saved 5X_L1200_small_molecule_test_textcoded.pyth (size: 34918713)
saved 5X_L1200_small_molecule_test.txt (size: 1096519)
saved 5X_L1200_small_molecule_test.prov (size: 9540)
(.venv) 

: 1

### Gene and proteins

In [ ]:
smtag-neo2xml -l1000 -y gene,protein -f gene_protein

In [9]:
smtag-convert2th -X5 -L1200 -c gene_protein -f 5X_L1200_gene_protein


loading from: gene_protein/train
[############################################################] 100% ...loaded 99.xml
754 examples; desired length:1200

[############################################################] 100% ...sampling example 753
length of the 754 examples selected:
1135 +/- 672 (min = 92, max = 4018)
Exec time for 'run': 75.584s
saving to 5X_L1200_gene_protein_train
saved 5X_L1200_gene_protein_train.pyth (size: 102845913)
saved 5X_L1200_gene_protein_train_textcoded.pyth (size: 149593913)
saved 5X_L1200_gene_protein_train.txt (size: 4705020)
saved 5X_L1200_gene_protein_train.prov (size: 44145)
(.venv) 

: 1

In [7]:
smtag-convert2th -X5 -L1200 -c gene_protein -f 5X_L1200_gene_protein -T


loading from: gene_protein/test
[############################################################] 100% ...loaded 99.xml
186 examples; desired length:1200

[############################################################] 100% ...sampling example 185
length of the 186 examples selected:
1267 +/- 926 (min = 113, max = 6835)
Exec time for 'run': 17.479s
saving to 5X_L1200_gene_protein_test
saved 5X_L1200_gene_protein_test.pyth (size: 25370713)
saved 5X_L1200_gene_protein_test_textcoded.pyth (size: 36902713)
saved 5X_L1200_gene_protein_test.txt (size: 1161228)
saved 5X_L1200_gene_protein_test.prov (size: 10085)
(.venv) 

: 1

### Subcellular components

In [ ]:
smtag-neo2xml -l1000 -y subcellular -f subcellular

In [ ]:
smtag-convert2th -X10 -L1200 -c gene_protein -f 10X_L1200_subcellular

In [ ]:
smtag-convert2th -X5 -L1200 -c gene_protein -f 10X_L1200_subcellular -T

### Cell line cell type

In [ ]:
smtag-neo2xml -l1000 -y cell -f cell

In [ ]:
smtag-convert2th -X10 -L1200 -c cell -f 10X_L1200_cell

In [ ]:
smtag-convert2th -X10 -L1200 -c cell -f 10X_L1200_cell -T

### Tissue

In [ ]:
smtag-neo2xml -l1000 -y tissue -f tissue

In [ ]:
smtag-convert2th -X10 -L1200 -c tissue -f 10X_L1200_tissue

In [ ]:
smtag-convert2th -X10 -L1200 -c tissue -f 10X_L1200_tissue -T

### Organism

In [ ]:
smtag-neo2xml -l1000 -y organism -f organism

In [ ]:
smtag-convert2th -X10 -L1200 -c organism -f 10X_L1200_organism

In [ ]:
smtag-convert2th -X10 -L1200 -c organism -f 10X_L1200_organism -T

### Experimental assays

In [ ]:
smtag-neo2xml -l100 -y exp_assay -f exp_assay

In [ ]:
smtag-convert2th -X10 -L1200 -c exp_assay -f 10X_L1200_exp_assay

In [ ]:
smtag-convert2th -X10 -L1200 -c exp_assay -f 10X_L1200_exp_assay -T

### Disease

Using a corpus in the brat format (`-b`) instead of the xml format.

In [10]:
smtag-convert2th -L1200 -X5 -b -c NCBI_disease

[############################################################] 100% ...loading examples


554 examples; desired length:1200

[############################################################] 100% ...sampling example 553
length of the 554 examples selected:
1276 +/- 418 (min = 283, max = 2655)
Exec time for 'run': 51.025s
saving to NCBI_disease_train_train
saved NCBI_disease_train_train.pyth (size: 75565913)
saved NCBI_disease_train_train_textcoded.pyth (size: 109913913)
saved NCBI_disease_train_train.txt (size: 3437570)
saved NCBI_disease_train_train.prov (size: 23040)
(.venv) 

: 1

In [11]:
smtag-convert2th -L1200 -X5 -b -c NCBI_disease -T

[############################################################] 100% ...loading examples


238 examples; desired length:1200

[############################################################] 100% ...sampling example 237
length of the 238 examples selected:
1294 +/- 401 (min = 505, max = 2837)
Exec time for 'run': 20.421s
saving to NCBI_disease_test_test
saved NCBI_disease_test_test.pyth (size: 32463513)
saved NCBI_disease_test_test_textcoded.pyth (size: 47219513)
saved NCBI_disease_test_test.txt (size: 1476790)
saved NCBI_disease_test_test.prov (size: 9500)
(.venv) 

: 1

### Intervention and measurements on geneproducts given reporter

Anonymize gene and proteins (-`A`) but do not anonymize reporters (`-AA`). Keep roles only for genes and proteins (`-s`).

In [ ]:
smtag-neo2xml -l1000 -y gene,protein -A gene,protein -AA reporter -s -f gene_protein_anonym_not_reporter

In [ ]:
smtag-convert2th -L1200 -X5 -c gene_reporter_anonym_but_not_reporter -f 5X_L1200_gene_protein_anonym_not_reporter

In [ ]:
smtag-convert2th -L1200 -X5 -c gene_reporter_anonym_but_not_reporter -f 5X_L1200_gene_protein_anonym_not_reporter -T

### Intervention and measurements on small molecules

### Reporter on geneproducts

Select enrich for protein and genes (`-y`) and reporters (`-r`). Remove all tags except genes and proteins (`-e`).

In [ ]:
smtag-neo2xml -l1000 -y gene,protein -r reporter -e -f gene_protein_reporter

In [ ]:
smtag-convert2th -L1200 -X5 -c reporter_gene_protein -f 5X_L1200_gene_protein_reporter

In [ ]:
smtag-convert2th -L1200 -X5 -c reporter_gene_protein -f 5X_L1200_gene_protein_reporter -T

## B. Moving training data from local `data4th/` to Amazon `EFS/smtag/data4th`

In [ ]:
zip data4th

In [ ]:
cd ../cloud

In [ ]:
scp -i basicLinuxAMI.pem ../smtag/data4th.zip ec2-user@smtag-web:/efs/smtag/data/

In [ ]:
ssh smtag-web

In [ ]:
cd /efs/smtag/

In [ ]:
unzip data4th.zip

## C. Training models

### Switch to GPU AMI

In [ ]:
ssh smtag-dl

In [ ]:
cd src/py-smtag

In [ ]:
source .venv/bin/activate

In [ ]:
python setup.py install

Logging on tensorboard

In [ ]:
tensorboard --logdir /efs/smtag/runs -port 6005 &

### Small molecule model

200 epoch with learning rate 0.001 and batch size 128

In [ ]:
smtag-meta -Z128 -E200 -R0.001 -o small_molecule -f 5X_L1200_small_molecule_train -w /efs/smtag

### Gene products

Scan hyperparameters:

In [ ]:
smtag-meta -E50 -o geneprod -H log_lr,log_batch_size -I 25 -f 5X_L1200_gene_protein_train

In [6]:
smtag-meta -E50 -o geneprod -H kernel,depth -I 25 -f 5X_L1200_gene_protein_train

opt[namebase]=5X_L1200_gene_protein_train
opt[learning_rate]=0.001
opt[epochs]=50
opt[minibatch_size]=2
opt[selected_features]=['geneprod']
opt[collapsed_features]=[]
opt[overlap_features]=[]
opt[features_as_input]=[]
opt[nf_table]=[8, 8, 8]
opt[pool_table]=[2, 2, 2]
opt[kernel_table]=[6, 6, 6]
opt[dropout]=0.1
opt[validation_fraction]=0.2
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [21]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading ./data4th/5X_L1200_gene_protein_train.pyth as features for the dataset.
Loading ./data4th/5X_L1200_gene_protein_train_textcoded.pyth as encoded text for the dataset.
Loading ./data4th/5X_L1200_gene_protein_train.txt for the original texts of the dataset.
Loading ./data4th/5X_L1200_gene_protein_train.prov as provenance info for the examples in the dataset.
Dataset dimensions:
3770 text examples of size 1240
0 input features (in-channels

Full training with selected hyperparameters:

In [ ]:
 smtag-meta -Z32 -E150 -R0.001 -kt 8,8,8 -o geneprod -f 5X_L1200_gene_protein_train

Scan thresholds for detection with the resulting model (ignore tokenization `-T`):

In [ ]:
smtag-eval -f 5X_L1200_gene_protein_train -m 5X_L1200_geneprod_<timestamp>.zip -S -T

## D. Benchmarking on the test set

Without tokenization (option `-T`)

In [ ]:
smtag-eval -f 5X_L1200_small_molecule_test -m 5X_L1200_small_molecule -T -w /efs/smtag

In [ ]:
smtag-eval -f 5X_L1200_gene_protein_test -m 5X_L1200_geneprod -T -w /efs/smtag

## E. Loading models into the rack

## F. Launch the SmartTag engine